In [1]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 5 hours 34 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,11 days
H2O cluster name:,H2O_from_python_oracle_ovyy7e
H2O cluster total nodes:,1
H2O cluster free memory:,6.245 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [3]:
import pandas as pd
from pandas import DataFrame

#from typing import Set, Any
#def remove_others(df: DataFrame, columns: Set[Any]):
#    cols_total: Set[Any] = set(df.columns)
#    diff: Set[Any] = cols_total - columns
#    df.drop(diff, axis=1, inplace=True)
    
### From Phishtank Import Malware urls

phishtank_trainpd=pd.read_csv("/home/oracle/notebooks/seim/online-valid.csv")
phishtank_trainds=phishtank_trainpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target'])
phishtank_trainds.head(3)
train=h2o.H2OFrame(phishtank_trainds)

### From Commom Crawl Import global urls :: Asume as verified=0 
#ccrawl_trainpd=pd.read_json("/home/oracle/notebooks/seim/tail-CC-MAIN-2018-43-index-.com.json")
#ccrawl_trainpd.head(2)
#ccrawl_trainds=remove_others(ccrawl_trainpd, {"url"})
#ccrawl_trainds.head(3)

phishtank_testpd=pd.read_csv("/home/oracle/notebooks/seim/online-valid.csv")
phishtank_testds=phishtank_testpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target']).head(5000)
phishtank_testds.head(3)
test=h2o.H2OFrame(phishtank_testds)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
# Import a sample binary outcome train/test set into H2O
#train = h2o.import_file("/home/oracle/notebooks/seim/online-valid.csv")
#test = h2o.import_file("/home/oracle/notebooks/seim/online-valid.csv").head(2000)

train.head()
train['verified'].describe()
test.head()
test['verified'].describe()


Rows:23849
Cols:1




,verified
type,int
mins,1.0
mean,1.0
maxs,1.0
sigma,0.0
zeros,0
missing,0
0,1.0
1,1.0
2,1.0


Rows:5000
Cols:1




,verified
type,int
mins,1.0
mean,1.0
maxs,1.0
sigma,0.0
zeros,0
missing,0
0,1.0
1,1.0
2,1.0


In [6]:
# Identify predictors and response
x = train.columns
y = "verified"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1999)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |████████████████████████████████████████████████████████| 100%
This H2OFrame is empty.
